<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/LLM_GPT2_Entropy_Semantic_Entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.3 MB/s eta 0:00:00


GPT2_FFT

In [ ]:
import torch
import os
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from datasets import Dataset
import torch.nn.functional as F
import random
import numpy as np

# Seed setting function
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility
seed = 50
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare 5 QA samples for training and validation
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Can you tell me who authored '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What planet is the biggest in our solar system?", "answer": "The largest planet is Jupiter."},
    {"question": "Who is the artist behind the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "How fast does light travel?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)

    # Clone input_ids into labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = tokenizer.eos_token_id  # Ignore question and padding tokens

    inputs["labels"] = labels
    return inputs

# Convert samples to dataset and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# Convert to PyTorch DataLoader
batch_size = 2

def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch])
    attention_mask = torch.tensor([item["attention_mask"] for item in batch])
    labels = torch.tensor([item["labels"] for item in batch])
    return input_ids, attention_mask, labels

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Cross-entropy loss function ignoring padding tokens
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model based on validation loss
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./gpt2-qa-best-loss-cml"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training and evaluation functions
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=10):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

        # Save best model based on validation loss
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=10)

# Load the best fine-tuned model for inference
model_name = "./gpt2-qa-best-loss-cml"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Training Loss: 4.4203, Validation Loss: 1.5726
Best model saved at epoch 1 with validation loss: 1.5726
Epoch 2/10, Training Loss: 0.9883, Validation Loss: 0.9799
Best model saved at epoch 2 with validation loss: 0.9799
Epoch 3/10, Training Loss: 0.7012, Validation Loss: 0.6840
Best model saved at epoch 3 with validation loss: 0.6840
Epoch 4/10, Training Loss: 0.4719, Validation Loss: 0.4885
Best model saved at epoch 4 with validation loss: 0.4885
Epoch 5/10, Training Loss: 0.2782, Validation Loss: 0.4139
Best model saved at epoch 5 with validation loss: 0.4139
Epoch 6/10, Training Loss: 0.2091, Validation Loss: 0.3706
Best model saved at epoch 6 with validation loss: 0.3706
Epoch 7/10, Training Loss: 0.1939, Validation Loss: 0.3273
Best model saved at epoch 7 with validation loss: 0.3273
Epoch 8/10, Training Loss: 0.1228, Validation Loss: 0.2700
Best model saved at epoch 8 with validation loss: 0.2700
Epoch 9/10, Training Loss: 0.1057, Validation Loss: 0.2148
Best model sa

In [ ]:
import torch
import os
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from datasets import Dataset
import torch.nn.functional as F
import random
import numpy as np

model_name = "./gpt2-qa-best-loss-cml"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Inference function with entropy-based uncertainty calculation
def generate_answer_with_entropy(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)

    # Generate with output scores to get logits at each generation step
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            output_scores=True,
            return_dict_in_generate=True
        )

    # Calculate entropy for each generated token (from the output scores)
    entropies = []
    for score in output.scores:
        # score shape: [batch_size, vocab_size]
        probs = F.softmax(score, dim=-1)
        entropy = -torch.sum(probs * torch.log(probs + 1e-10), dim=-1)  # entropy for each sample in the batch
        entropies.append(entropy)
    # Average entropy over generated tokens
    mean_entropy = torch.mean(torch.stack(entropies))

    # Decode generated text and extract answer part
    answer = tokenizer.decode(output.sequences[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer, mean_entropy.item()

# Example inference
question = "What is the capital of France?"
answer, uncertainty = generate_answer_with_entropy(question)
print(f"Q: {question}\nA: {answer}\nMean Entropy (Uncertainty): {uncertainty:.4f}")


Q: What is the capital of France?
A: The capital of France is Paris.
Mean Entropy (Uncertainty): 0.7557


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch.nn.functional as F
import numpy as np
from sklearn.cluster import KMeans

# Load GPT-2 model and tokenizer
model_name = "./gpt2-qa-best-loss-cml"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

def generate_answer_with_semantic_entropy(question, n_clusters=5):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)
    input_length = inputs['input_ids'].shape[1]

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            output_scores=True,
            return_dict_in_generate=True,
            output_hidden_states=True
        )

    # Decode generated answer
    full_text = tokenizer.decode(output.sequences[0], skip_special_tokens=True)
    answer = full_text.split("Answer:")[-1].strip()

    # Extract hidden states from the last layer; ensure it's a tensor
    last_hidden = output.hidden_states[-1]
    if isinstance(last_hidden, tuple):
        last_hidden = last_hidden[-1]

    # Use only the generated tokens (excluding the input prompt)
    generated_hidden = last_hidden[0, input_length:, :]  # shape: [num_generated_tokens, hidden_dim]
    num_tokens = generated_hidden.shape[0]

    # Guard: if no new tokens were generated, return zero entropy
    if num_tokens == 0:
        return answer, 0.0

    # Convert hidden states to numpy array for clustering
    generated_hidden_np = generated_hidden.cpu().numpy()
    k = min(n_clusters, num_tokens)
    kmeans = KMeans(n_clusters=k, random_state=0)
    cluster_labels = kmeans.fit_predict(generated_hidden_np)

    # Compute the probability distribution over clusters
    counts = np.bincount(cluster_labels, minlength=k)
    probs = counts / np.sum(counts)

    # Compute semantic entropy
    epsilon = 1e-10
    semantic_entropy = -np.sum(probs * np.log(probs + epsilon))

    return answer, semantic_entropy

# Example usage:
question = "What is the capital of France?"
answer, sem_entropy = generate_answer_with_semantic_entropy(question)
print(f"Q: {question}\nA: {answer}\nSemantic Entropy: {sem_entropy:.4f}")


Q: What is the capital of France?
A: The capital of France is Paris.
Semantic Entropy: 0.0000


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch.nn.functional as F
import numpy as np
from sklearn.cluster import KMeans

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_answer_with_semantic_entropy(question, n_clusters=5):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)
    input_length = inputs['input_ids'].shape[1]

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            output_scores=True,
            return_dict_in_generate=True,
            output_hidden_states=True
        )

    # Decode generated answer
    full_text = tokenizer.decode(output.sequences[0], skip_special_tokens=True)
    answer = full_text.split("Answer:")[-1].strip()

    print('input_ids:', inputs['input_ids'].shape)
    print('output.sequences:', output.sequences.shape)
    print('output.hidden_states:', len(output.hidden_states), len(output.hidden_states[0]))
    print('output.hidden_states:', output.hidden_states[0][0].shape)
    # Extract hidden states from the last layer; ensure it's a tensor
    last_hidden = output.hidden_states[-1]
    print('last_hidden[-1]:', last_hidden[-1].shape)
    if isinstance(last_hidden, tuple):
        last_hidden = last_hidden[-1]

    # Use only the generated tokens (excluding the input prompt)
    generated_hidden = last_hidden[0, input_length:, :]  # shape: [num_generated_tokens, hidden_dim]
    num_tokens = generated_hidden.shape[0]
    print('num_tokens:', num_tokens)
    # Guard: if no new tokens were generated, return zero entropy
    if num_tokens == 0:
        return answer, 0.0

    # Convert hidden states to numpy array for clustering
    generated_hidden_np = generated_hidden.cpu().numpy()
    k = min(n_clusters, num_tokens)
    kmeans = KMeans(n_clusters=k, random_state=0)
    cluster_labels = kmeans.fit_predict(generated_hidden_np)

    # Compute the probability distribution over clusters
    counts = np.bincount(cluster_labels, minlength=k)
    probs = counts / np.sum(counts)

    # Compute semantic entropy
    epsilon = 1e-10
    semantic_entropy = -np.sum(probs * np.log(probs + epsilon))

    return answer, semantic_entropy

# Example usage:
question = "What is the capital of France?"
answer, sem_entropy = generate_answer_with_semantic_entropy(question)
print(f"Q: {question}\nA: {answer}\nSemantic Entropy: {sem_entropy:.4f}")


input_ids: torch.Size([1, 11])
output.sequences: torch.Size([1, 61])
output.hidden_states: 50 13
output.hidden_states: torch.Size([1, 11, 768])
last_hidden[-1]: torch.Size([1, 1, 768])
num_tokens: 0
Q: What is the capital of France?
A: The capital of France
Semantic Entropy: 0.0000


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch.nn.functional as F
import numpy as np
from sklearn.cluster import KMeans

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_answer_with_semantic_entropy(question, n_clusters=5):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)
    input_length = inputs['input_ids'].shape[1]

    # Generate output (without hidden states, as they might not include the generated tokens)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True
        )

    generated_ids = output.sequences  # shape: [batch_size, sequence_length]
    generated_sequence_length = generated_ids.shape[1]
    print(f"Input length: {input_length}, Generated sequence length: {generated_sequence_length}")

    # Decode full generated text
    full_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    answer = full_text.split("Answer:")[-1].strip()

    # Now, run a forward pass on the entire generated sequence to get hidden states
    with torch.no_grad():
        outputs = model(generated_ids, output_hidden_states=True)
    # Extract the hidden states from the last layer: shape [batch_size, sequence_length, hidden_dim]
    last_hidden = outputs.hidden_states[-1]
    # Get the hidden states corresponding to the tokens generated after the prompt
    generated_hidden = last_hidden[0, input_length:, :]
    num_tokens = generated_hidden.shape[0]
    print("num_tokens:", num_tokens)

    if num_tokens == 0:
        return answer, 0.0

    # Convert hidden states to numpy array and perform clustering
    generated_hidden_np = generated_hidden.cpu().numpy()
    k = min(n_clusters, num_tokens)
    kmeans = KMeans(n_clusters=k, random_state=0)
    cluster_labels = kmeans.fit_predict(generated_hidden_np)

    # Compute probability distribution over clusters
    counts = np.bincount(cluster_labels, minlength=k)
    probs = counts / np.sum(counts)

    # Compute semantic entropy
    epsilon = 1e-10
    semantic_entropy = -np.sum(probs * np.log(probs + epsilon))

    return answer, semantic_entropy

# Example usage:
question = "What is the capital of France?"
answer, sem_entropy = generate_answer_with_semantic_entropy(question)
print(f"Q: {question}\nA: {answer}\nSemantic Entropy: {sem_entropy:.4f}")


Input length: 11, Generated sequence length: 61
num_tokens: 50
Q: What is the capital of France?
A: The capital of France
Semantic Entropy: 1.5363
